## Heat flow equation

We create a finite difference solution to the following system using a Crank-Nicholson finite difference scheme:

$u_t=u_{xx},\quad x\in[a,b],\,t\in(0,T]$

$u(x,0)=u_0(x)$

$h_a(t)=c_a(t)u(a,t)+d_a(t)u_x(a,t)$

$h_b(t)=c_b(t)u(b,t)+d_b(t)u_x(b,t)$.

In [ ]:
def heat_equation(a, b, T, N_x, N_t, u_0, c_a, d_a, h_a, c_b, d_b, h_b):
def crank_nicolson(nu, k, h, f, x, u_0t, u_1t):
    """Performs the Crank Nicolson method for the PDE from the PDF, with coefficients specified as parameters."""
    
    lamby = nu * k / h / h / 2
    
    x = np.linspace(a, b, N_x)
    
    # construct A
    A = np.zeros((len(x) - 1, len(x) - 1))
    np.fill_diagonal(A, 1 - 2 * lamby)
    np.fill_diagonal(A[1:], lamby)
    np.fill_diagonal(A[:,1:], lamby)
    
    # construct B
    B = np.zeros((len(x), len(x)))
    np.fill_diagonal(B, 1 + 2 * lamby)
    np.fill_diagonal(B[1:], -lamby)
    np.fill_diagonal(B[:,1:], -lamby)
    
    # modify B to leave boundary conditions intact
    B[0, 1] = 0
    B[0, 0] = 1
    B[-1, -2] = 0
    B[-1, -1] = 1
    
    # get B's inverse
    B_inv = np.linalg.inv(B)
    
    # get B^{-1}A
    mult = np.dot(B_inv, A)
    
    # set up initial condition for t=0
    f_x0 = f(x)
    f_x0[0] = u_0t
    f_x0[-1] = u_1t
    
    # temporal iteration
    yous = [f_x0]
    
    for i in range(int(1 / k)):
        yous.append(np.dot(mult, yous[-1]))
    
    return yous